In [14]:
from pypower.api import loadcase, runopf, ppoption
from pypower.idx_brch import *
from pypower.idx_bus import *
from pypower.idx_gen import *
from pypower.idx_cost import *
from pypower.ext2int import ext2int
import oct2py
import numpy as np
import math
import os
from tqdm import tqdm
from oct2py import Oct2Py
octave = Oct2Py()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import pickle
import shutil
import cvxpy as cp
from makeYbus import makeYbus
if shutil.which('latex') is None:
    LATEX_AVAILABLE = False
else:
    LATEX_AVAILABLE = True
if LATEX_AVAILABLE:
    plt.rcParams['font.size'] = 14
    plt.rcParams['text.usetex'] = True
    plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,amssymb}'
    
MAX_BUS = 1000 # upper limit of number of buses in cases to be considered

In [8]:
# get all cases in current directory
current_directory = os.getcwd()
all_files_and_directories = os.listdir(current_directory)
case_files = [os.path.join(current_directory, f) for f in all_files_and_directories if f.endswith('.m') and os.path.isfile(os.path.join(current_directory, f))]

cases, casenames = [], []
cases_full, casenames_full = [], []
for cf in (t:=tqdm(case_files)):
    octave.source(os.path.basename(cf))
    cname = os.path.basename(cf).split('.')[0]
    num_buses = None
    for ci in cname.split('_'):
        if 'case' in ci:
            num_buses = int(''.join([chr for chr in ci.replace('case','',1) if chr.isdigit()]))
    if num_buses <= MAX_BUS:
        cases.append(loadcase(octave.feval(cname)))
        casenames.append(cname)
    cases_full.append(loadcase(octave.feval(cname)))
    casenames_full.append(cname)
    t.set_description(f"Loaded {os.path.basename(cf)}")

Loaded pglib_opf_case793_goc.m: 100%|██████████████████████████████████████████████████████████████████████████| 66/66 [00:55<00:00,  1.19it/s]


In [43]:
# use a specific case to test out algorithms
CASE_IDX = 3
curcase = cases[CASE_IDX]
curcase = ext2int(curcase) # convert from extrnal to internal numbering and automatically remove inactive elements
curname = casenames[CASE_IDX]   

In [44]:
# get bus matrices
Ybus, Yf, Yt, Cf, Ct = makeYbus(curcase['baseMVA'],curcase['bus'],curcase['branch'])
Ybus, Yf, Yt, Cf, Ct = Ybus.todense(), Yf.todense(), Yt.todense(), Cf.todense(), Ct.todense()
Yf_full, Yt_full = Cf.T @ Yf, Ct.T @ Yt
Yf_abs, Yt_abs = np.abs(Yf_full), np.abs(Yt_full)

In [70]:
# reckon the elements which are to be nonzero
v_len = Ybus.shape[0]
term_required = np.zeros((v_len,v_len))

for fb in np.arange(v_len):
    for tb in np.arange(v_len):
        onehot_from, onehot_to = np.eye(v_len)[fb], np.eye(v_len)[tb]
        Sfvec = np.diag(Cf @ onehot_from) @ (np.conj(Yf) @ onehot_to)
        if np.any(np.abs(Sfvec)):
            print(f"Detected nonzero Sf coeffs, from bus: {fb+1}, to bus: {tb+1}.\n")
for fb in np.arange(v_len):
    for tb in np.arange(v_len):
        onehot_from, onehot_to = np.eye(v_len)[fb], np.eye(v_len)[tb]
        Stvec = np.diag(Ct @ onehot_to) @ (np.conj(Yt) @ onehot_from)
        if np.any(np.abs(Stvec)):
            print(f"Detected nonzero St coeffs, from bus: {fb+1}, to bus: {tb+1}.\n")
    

Detected nonzero Sf coeffs, from bus: 1, to bus: 1.

Detected nonzero Sf coeffs, from bus: 1, to bus: 2.

Detected nonzero Sf coeffs, from bus: 1, to bus: 3.

Detected nonzero Sf coeffs, from bus: 1, to bus: 4.

Detected nonzero Sf coeffs, from bus: 1, to bus: 5.

Detected nonzero St coeffs, from bus: 1, to bus: 2.

Detected nonzero St coeffs, from bus: 2, to bus: 2.

Detected nonzero St coeffs, from bus: 3, to bus: 2.

Detected nonzero St coeffs, from bus: 4, to bus: 2.

Detected nonzero St coeffs, from bus: 5, to bus: 2.



In [59]:
import pickle
with open('pglib_opf_case5_pjm_solved.pkl','rb') as file:
    data = pickle.load(file)

In [65]:
for k in data.keys():
    print(k)

version
baseMVA
areas
bus
gen
gencost
branch
order
om
x
mu
f
var
lin
nln
et
success
raw


In [67]:
data['raw']

{'xr': array([ 4.89352552e-02, -1.28220006e-02, -9.76909751e-03,  0.00000000e+00,
         6.26634252e-02,  1.07761731e+00,  1.08406435e+00,  1.09999959e+00,
         1.06413694e+00,  1.06907035e+00,  3.99999780e-01,  1.69999967e+00,
         3.24498200e+00,  2.24830275e-06,  4.70693727e+00,  2.99998188e-01,
         1.27499819e+00,  3.89999384e+00, -1.08018323e-01, -1.65038887e+00]),
 'pimul': array([-1.69350821e+03, -2.65499076e+03, -3.00000001e+03, -3.97120867e+03,
        -1.00000004e+03, -3.57041278e+01, -3.67385618e+01, -1.05113358e+01,
         6.26656852e-06,  1.21888846e-05,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -6.13108461e+03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  9.23329760e